- モデル計算

# Library

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# model
import lightgbm as lgb

# shap
import shap

In [ ]:
# my_tool
import sys
sys.path.append('../../')
import importlib
from utils import my_ds_tools as mdt
importlib.reload(mdt)

# Config

In [ ]:
VERSION = 'v1'
USAGE = 'tree'
INPUT_DIR = '../intermediate/'

In [ ]:
Y_COL = 'Survived'
DROP_COLS = ['PassengerId']

# Load data

In [ ]:
train_df = pd.read_csv(INPUT_DIR + 'train_for_' + USAGE + '_' + VERSION + '.csv')
test_df = pd.read_csv(INPUT_DIR + 'test_for_' + USAGE + '_' + VERSION + '.csv')

# Preprocess

In [ ]:
feature_cols = [
#     'Unnamed: 0',
#     'PassengerId',
    'Pclass',
#     'Name',
    'Sex',
#     'Age',
#     'SibSp',
#     'Parch', 
#     'Ticket', 
#     'Fare',
#     'Cabin',
    'Embarked',
#     'name_title',
    'ticket_length',
    'cabin_length',
    'cabin_initial',
    'fixed_age',
#     'age_rank',
    'fixed_fare',
    'is_alone',
#     'Survived'
]
# train_df.columns

In [ ]:
x = train_df.loc[:, feature_cols]
y = train_df.loc[:, Y_COL]

In [ ]:
prepro_tool = mdt.DS_PREPROCESS(x, y)

In [ ]:
prepro_tool.show_desc_features()

In [ ]:
prepro_x = prepro_tool.label_encode_x()
prepro_tool.check_null_x(prepro_x)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(prepro_x, y)

# Calculate model

In [ ]:
prepro_x.head()

In [ ]:
models = ['lgb', 'rbf_svm', 'linear_svm', 'logit', 'dt', 'rf', 'nb', 'knn', 'vote']
model_tool = mdt.DS_MODEL()
for m in models:
    print(m)
#     model_tool.class_fit_predict(x_train, x_test, y_train, y_test, m)
    model_tool.class_fit_predict_cv(m, prepro_x, y)
    print('---------------------------------')

# EDA by SHAP

In [ ]:
fitted_models = model_tool.get_estimators()
fitted_models

In [ ]:
shap_tool = mdt.DS_SHAP(fitted_models['lgb'], x_train)

In [ ]:
shap_tool.show_summary_plot()

In [ ]:
shap_tool.show_all_dependence_plot()